# Insurance Claims Processing with Semantic Kernel Agents

This notebook demonstrates how to leverage Azure OpenAI's Assistant API with Semantic Kernel for intelligent claims processing. We'll create a multi-agent system that can:

1. Analyze claim documents using File Search capabilities
2. Coordinate between multiple specialized agents using Group Chat
3. Generate actionable insights and recommendations

## Scenario: Commercial Property Hurricane Damage Claim

We're processing claims for a retail chain (SmartMart Electronics) that experienced hurricane damage across multiple Florida locations. The system will help:
- Analyze damage reports and supporting documentation
- Verify policy compliance
- Assess risk factors
- Generate recommendations

## Setup and Dependencies

In [ ]:
!pip install semantic-kernel==1.17.0 python-dotenv aiofiles nest_asyncio azure-search-documents pyperclip


## Import and Setup

In [39]:
# Import required libraries
import asyncio
import os
from pathlib import Path
from typing import List
from dataclasses import dataclass
from dotenv import load_dotenv
import logging


from semantic_kernel.kernel import Kernel
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.streaming_annotation_content import StreamingAnnotationContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import (
    KernelFunctionSelectionStrategy,
)
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import (
    KernelFunctionTerminationStrategy,
)
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel


# Enable notebook async support
import nest_asyncio
nest_asyncio.apply()

load_dotenv()
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-01-preview")


In [54]:
@dataclass
class AzureConfig:
    """Configuration for Azure OpenAI Assistant."""
    api_key: str
    endpoint: str
    deployment_name: str
    api_version: str = "2024-10-01-preview"

    @classmethod
    def from_env(cls) -> "AzureConfig":
        """Load configuration from environment variables."""
        load_dotenv()
        
        # Get environment variables
        api_key = os.getenv("AZURE_OPENAI_API_KEY")
        endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        deployment_name = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME")
        
        if not all([api_key, endpoint, deployment_name]):
            missing = [var for var, val in {
                "AZURE_OPENAI_API_KEY": api_key,
                "AZURE_OPENAI_ENDPOINT": endpoint,
                "AZURE_OPENAI_CHAT_COMPLETION_DEPLOYED_MODEL_NAME": deployment_name,
            }.items() if not val]
            raise ValueError(f"Missing required environment variables: {', '.join(missing)}")
            
        return cls(
            api_key=api_key,
            endpoint=endpoint,
            deployment_name=deployment_name
        )

# Initialize config at module level
config = AzureConfig.from_env()

Let's create our sample insurance documents:

In [53]:
def save_sample_documents():
    """Create and save sample insurance claim documents."""
    documents = {
        "claim_report.txt": """
        Claim ID: HURFL-2024-0113
        Date Filed: December 13, 2024
        Store Location: SmartMart Electronics #FL-156
        Address: 2850 Commercial Blvd, Tampa, FL 33618

        Initial Damage Report:
        Store manager Michael Chen reported severe water damage following Hurricane Ian...
        """,
        
        "policy_details.txt": """
        Policy Number: CP-FL-789201
        Policyholder: SmartMart Electronics Inc.
        Policy Type: Commercial Property Insurance
        Term: January 1, 2024 - December 31, 2024...
        """,
        
        "weather_report.txt": """
        NOAA Storm Report
        Date: December 12-13, 2024
        Region: Tampa Bay Area, Florida
        Hurricane Ian Data:
        - Category: 3 Hurricane
        - Peak Wind Speed: 125 mph...
        """
    }
    
    base_directory = Path.cwd() / "insurance_docs"
    base_directory.mkdir(exist_ok=True)
    
    for filename, content in documents.items():
        filepath = base_directory / filename
        filepath.write_text(content)
        
    return list(documents.keys())

document_files = save_sample_documents()

## Agent Setup and Roles

We'll create three specialized agents to collaborate on claims processing:

1. **Claims Analyst Agent**: Reviews initial claims and coordinates analysis
2. **Policy Verifier Agent**: Checks policy compliance and coverage details
3. **Risk Assessor Agent**: Evaluates risks and recommends actions

Each agent will have specific responsibilities and can access the document store through File Search capabilities.

In [55]:
async def create_specialized_agents(kernel: Kernel, config: AzureConfig):
    # Claims anaylsis using File Search for Analysis
    claims_analyst = await AzureAssistantAgent.create(
        kernel=kernel,
        deployment_name=config.deployment_name,
        endpoint=config.endpoint,
        api_key=config.api_key,
        name="Claims_Analyst",
        instructions="""You are an expert insurance claims analyst. 
        First analyze all provided documents to extract:
        - Claim details from claim reports
        - Coverage details from policy documents
        - Supporting evidence from additional documents
        
        Always cite specific sections when making assessments.
        Use the search tool to find relevant information.""",
        enable_file_search=True,
        vector_store_filenames=[str(Path.cwd() / "insurance_docs" / f) for f in document_files]
    )
    
    # Policy Verifier - Specializes in policy interpretation
    policy_verifier = ChatCompletionAgent(
        service_id="azure-chat",  # Use the same service ID as defined in setup_kernel
        kernel=kernel,
        name="Policy_Verifier",
        instructions="""You are an insurance policy verification specialist. Your responsibilities:
        1. Verify coverage details and policy limits
        2. Identify relevant exclusions or special conditions
        3. Confirm compliance with policy requirements
        4. Flag any potential coverage issues
        
        Always reference specific policy sections and requirements."""
    )
    
    # Risk Assessor - Evaluates risks and provides recommendations
    risk_assessor = ChatCompletionAgent(
        service_id="azure-chat",  # Use the same service ID as defined in setup_kernel
        kernel=kernel,
        name="Risk_Assessor",
        instructions="""You are an insurance risk assessment specialist. Your responsibilities:
        1. Evaluate severity and impact of reported damages
        2. Assess compliance with building codes and safety requirements
        3. Identify potential fraud indicators
        4. Recommend risk mitigation measures
        
        Provide clear evidence-based risk ratings and specific recommendations."""
    )
    
    return claims_analyst, policy_verifier, risk_assessor

## Group Chat Coordination

Now we'll set up the Agent Group Chat to coordinate the collaboration between our specialized agents. We'll define:
1. Selection strategy for determining which agent should respond
2. Termination strategy for completing the analysis
3. Workflow patterns for different types of claims analysis

In [56]:
async def setup_kernel(config: AzureConfig) -> Kernel:
    """Initialize kernel with required services."""
    kernel = Kernel()
    
    # Add Azure OpenAI Chat Completion service
    # Use a single service ID that will be shared by all agents
    chat_service = AzureChatCompletion(
        deployment_name=config.deployment_name,
        endpoint=config.endpoint,
        api_key=config.api_key,
        api_version=config.api_version,
        service_id="azure-chat"  # Single service ID for all chat completion agents
    )
    kernel.add_service(chat_service)
    
    return kernel

In [70]:
def create_history_string(messages: List[ChatMessageContent]) -> str:
    """Create a formatted history string from messages."""
    history = []
    for msg in messages:
        if msg.role == AuthorRole.ASSISTANT:
            history.append(f"{msg.name}: {msg.content}")
        else:
            history.append(f"User: {msg.content}")
    return "\n".join(history[-5:])  # Keep last 5 messages for context

In [71]:
async def setup_group_chat(kernel: Kernel, claims_analyst, policy_verifier, risk_assessor):
    """Configure group chat coordination between agents."""
    
    selection_strategy = KernelFunctionSelectionStrategy(
        kernel=kernel,
        function=KernelFunctionFromPrompt(
            function_name="selection",
            prompt="""Determine next agent based on current stage of analysis.
            
            Current agents:
            - Claims_Analyst: Reviews documents, provides initial and final assessment
            - Policy_Verifier: Verifies coverage and compliance
            - Risk_Assessor: Evaluates risks and recommendations
            
            Analysis flow:
            1. Claims_Analyst (first) -> Document review
            2. Policy_Verifier -> Coverage verification
            3. Risk_Assessor -> Risk evaluation
            4. Claims_Analyst (last) -> Final recommendation
            
            Current conversation:
            {{$history}}
            
            Return only the name of the next agent to respond."""
        ),
        result_parser=lambda result: str(result.value[0]) if result.value is not None else "Claims_Analyst",
        history_variable_name="history",
        history_reducer=create_history_string
    )
    
    termination_strategy = KernelFunctionTerminationStrategy(
        kernel=kernel,
        function=KernelFunctionFromPrompt(
            function_name="termination",
            prompt="""Check if analysis is complete by verifying:
            1. Claims_Analyst has reviewed documents
            2. Policy_Verifier has verified coverage
            3. Risk_Assessor has evaluated risks
            4. Claims_Analyst has provided final recommendation
            
            Current conversation:
            {{$history}}
            
            Return 'COMPLETE' only if all steps are done."""
        ),
        result_parser=lambda result: str(result.value[0]).strip().upper() == "COMPLETE",
        history_variable_name="history",
        history_reducer=create_history_string,
        maximum_iterations=10
    )
    
    return AgentGroupChat(
        agents=[claims_analyst, policy_verifier, risk_assessor],
        selection_strategy=selection_strategy,
        termination_strategy=termination_strategy
    )
    
    return chat

In [72]:
async def verify_services(kernel: Kernel):
    """Verify that required services are properly configured."""
    try:
        chat_service = kernel.get_service("azure-chat")
        logger.info("Chat completion service verified")
        return True
    except Exception as e:
        logger.error(f"Service verification failed: {str(e)}")
        raise RuntimeError("Required services not properly configured") from e

## Claims Analysis Workflow

Let's implement the main workflow for processing claims. This will:
1. Initialize the document analysis using File Search
2. Coordinate agent collaboration through Group Chat
3. Generate a comprehensive claim assessment

In [73]:
async def process_claim(chat: AgentGroupChat, claim_id: str):
    """Process an insurance claim through multi-agent analysis."""
    
    stages = {
        "document_review": False,
        "coverage_verification": False,
        "risk_assessment": False,
        "final_recommendation": False
    }
    
    last_progress_string = ""
    
    def print_progress():
        nonlocal last_progress_string
        progress_string = "\nCurrent Progress:"
        for stage, completed in stages.items():
            status = "✅" if completed else "⭕"
            progress_string += f"\n{status} {stage.replace('_', ' ').title()}"
        
        # Only print if progress has changed
        if progress_string != last_progress_string:
            print(progress_string)
            last_progress_string = progress_string

    print(f"🔄 Starting analysis for claim {claim_id}\n")
    print_progress()

    async for response in chat.invoke_stream():
        if isinstance(response, StreamingAnnotationContent):
            logger.debug(f"Received citation from {response.file_id}")
            print(f"\n📄 Citation from {response.file_id}:")
            print(f"'{response.quote}'\n")
            stages["document_review"] = True
            print_progress()
        else:
            # Process agent response
            if response.content:
                if "Policy_Verifier" in str(response.name):
                    stages["coverage_verification"] = True
                elif "Risk_Assessor" in str(response.name):
                    stages["risk_assessment"] = True
                elif "Claims_Analyst" in str(response.name) and "final recommendation" in response.content.lower():
                    stages["final_recommendation"] = True
                
                print(f"\n👤 {response.name} is analyzing...")
                print(response.content)
                print_progress()

    logger.info("Claim analysis complete")
    print("\n✅ Claim analysis complete!")

## Example Usage

Let's demonstrate the system with our sample hurricane damage claim:

In [74]:
async def main():
    try:
        # Initialize configuration and kernel
        config = AzureConfig.from_env()
        kernel = await setup_kernel(config)
        
        print("Creating sample documents...")
        document_files = save_sample_documents()
        print(f"Created sample documents: {document_files}")
        
        print("\nInitializing agents...")
        claims_analyst, policy_verifier, risk_assessor = await create_specialized_agents(kernel, config)
        
        print("\nSetting up group chat...")
        try:
            chat = await setup_group_chat(kernel, claims_analyst, policy_verifier, risk_assessor)
        except Exception as e:
            logger.error(f"Error setting up group chat: {str(e)}")
            raise
        
        print("\nProcessing claim HURFL-2024-0113...")
        await process_claim(chat, "HURFL-2024-0113")
        
    except Exception as e:
        logger.error(f"Error during execution: {str(e)}", exc_info=True)
        raise
    finally:
        print("\nCleaning up resources...")
        if 'claims_analyst' in locals():
            try:
                await claims_analyst.delete()
            except Exception as e:
                logger.warning(f"Error cleaning up claims analyst: {str(e)}")
        if 'chat' in locals():
            try:
                await chat.reset()
            except Exception as e:
                logger.warning(f"Error resetting chat: {str(e)}")

# Run the analysis (explicitly call main)
await main()


Creating sample documents...
Created sample documents: ['claim_report.txt', 'policy_details.txt', 'weather_report.txt']

Initializing agents...


INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com//openai/files?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com//openai/files?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com//openai/files?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com//openai/vector_stores?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com//openai/assistants?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method called, invoking function: , selection
INFO:semantic_kernel.functions.kernel_function:Function selection invoking.



Setting up group chat...

Processing claim HURFL-2024-0113...
🔄 Starting analysis for claim HURFL-2024-0113


Current Progress:
⭕ Document Review
⭕ Coverage Verification
⭕ Risk Assessment
⭕ Final Recommendation


INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=3, prompt_tokens=137, total_tokens=140, completion_tokens_details=None)
INFO:semantic_kernel.functions.kernel_function:Function selection succeeded.
INFO:semantic_kernel.functions.kernel_function:Function completed. Duration: 0.572908s
INFO:semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy:Kernel Function Selection Strategy next method completed: , selection, result: [ChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-AeR3trheSba0mYv27htooTPZTZvEi', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Policy_Verifier', refusal=None, role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate


👤 Policy_Verifier is analyzing...
Sure

Current Progress:
⭕ Document Review
✅ Coverage Verification
⭕ Risk Assessment
⭕ Final Recommendation

👤 Policy_Verifier is analyzing...
,

👤 Policy_Verifier is analyzing...
 I

👤 Policy_Verifier is analyzing...
 can

👤 Policy_Verifier is analyzing...
 assist

👤 Policy_Verifier is analyzing...
 you

👤 Policy_Verifier is analyzing...
 with

👤 Policy_Verifier is analyzing...
 that

👤 Policy_Verifier is analyzing...
.

👤 Policy_Verifier is analyzing...
 Please

👤 Policy_Verifier is analyzing...
 provide

👤 Policy_Verifier is analyzing...
 the

👤 Policy_Verifier is analyzing...
 details

👤 Policy_Verifier is analyzing...
 of

👤 Policy_Verifier is analyzing...
 the

👤 Policy_Verifier is analyzing...
 insurance

👤 Policy_Verifier is analyzing...
 policy

👤 Policy_Verifier is analyzing...
 you

👤 Policy_Verifier is analyzing...
 would

👤 Policy_Verifier is analyzing...
 like

👤 Policy_Verifier is analyzing...
 me

👤 Policy_Verifier is analyzing...
 to



...

INFO:semantic_kernel.agents.open_ai.open_ai_assistant_base:Run step completed with ID: step_vQvPtgZNWu84vUOL5CTBWgtZ
INFO:semantic_kernel.agents.open_ai.open_ai_assistant_base:Run completed with ID: run_x8pil7KeCVsdoY5OcyPwYOFo



👤 Claims_Analyst is analyzing...
 to

👤 Claims_Analyst is analyzing...
 ask

👤 Claims_Analyst is analyzing...
!


INFO:httpx:HTTP Request: GET https://fsunavala-openai-eus.openai.azure.com//openai/threads/thread_aH3igGVBZOIjca9zqdAea5VD/messages/msg_Qg9fMq2KWFGtGE7EaiX1ZmJA?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.agents.strategies.termination.termination_strategy:Evaluating termination criteria for e497a1b8-ac5e-4459-8a87-0814b3ee2d7d
INFO:semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy:should_agent_terminate, function invoking: `termination`
INFO:semantic_kernel.functions.kernel_function:Function termination invoking.
INFO:httpx:HTTP Request: POST https://fsunavala-openai-eus.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
INFO:semantic_kernel.connectors.ai.open_ai.services.open_ai_handler:OpenAI usage: CompletionUsage(completion_tokens=132, prompt_tokens=3514, total_tokens=3646, completion_tokens_details=None)
INFO:semantic_kernel.functions.kernel_function:Function t


✅ Claim analysis complete!

Cleaning up resources...


INFO:httpx:HTTP Request: DELETE https://fsunavala-openai-eus.openai.azure.com//openai/threads/thread_aH3igGVBZOIjca9zqdAea5VD?api-version=2024-10-01-preview "HTTP/1.1 200 OK"
